In [1]:
from kendama_env import KendamaEnv
import numpy as np 
import time
env = KendamaEnv()


start_time = time.time()
while 1:
    current_time = time.time()
    elapsed_time = current_time - start_time

    if elapsed_time > 1.0/env.dt:
        start_time = time.time()
        # action = [list(np.random.normal(scale = 0,loc=1,size = 3)),list(np.random.normal(scale = 10,size = 3))]
        action = [[0,0,0],[0,0,0]]
        ob, reward, done, _ = env.step(action)

error: Not connected to physics server.